In [1]:
import numpy as np
import pandas as pd
import scipy
from scipy import stats
import suncalc

import dask.dataframe as dd
from pathlib import Path
from tqdm import tqdm
import re

import datetime as dt

import matplotlib.pyplot as plt
from matplotlib import colors
import soundfile as sf
import matplotlib.patches as patches

In [2]:
import sys

sys.path.append("../src")
sys.path.append("../src/activity")

In [3]:
import subsampling as ss
import activity_assembly as actvt
import bout.assembly as bt
import comparison.data_assembly as comp
import comparison.plot as complot
from core import SITE_NAMES

from cli import get_file_paths
import plot
import pipeline

In [4]:
avail = np.arange(0, 180, 2) + 2
reset_3 = avail[np.where((3*60 % avail) == 0)[0]]
reset_4 = avail[np.where((4*60 % avail) == 0)[0]]
reset_6 = avail[np.where((6*60 % avail) == 0)[0]]
reset_12 = avail[np.where((12*60 % avail) == 0)[0]]
reset_24 = avail[np.where((24*60 % avail) == 0)[0]]

In [5]:
reset_3

array([  2,   4,   6,  10,  12,  18,  20,  30,  36,  60,  90, 180])

In [6]:
reset_4

array([  2,   4,   6,   8,  10,  12,  16,  20,  24,  30,  40,  48,  60,
        80, 120])

In [7]:
reset_6

array([  2,   4,   6,   8,  10,  12,  18,  20,  24,  30,  36,  40,  60,
        72,  90, 120, 180])

In [8]:
reset_12

array([  2,   4,   6,   8,  10,  12,  16,  18,  20,  24,  30,  36,  40,
        48,  60,  72,  80,  90, 120, 144, 180])

In [9]:
reset_24

array([  2,   4,   6,   8,  10,  12,  16,  18,  20,  24,  30,  32,  36,
        40,  48,  60,  72,  80,  90,  96, 120, 144, 160, 180])

In [10]:
f = 10
fact_ = reset_12[np.where((reset_12%f) == 0)[0]]
fact_

array([ 10,  20,  30,  40,  60,  80,  90, 120, 180])

In [11]:
step = 1/10
step_by = np.arange(step, 1, step)
# step_by = step_by[step_by>=(1/6)]
step_by

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [12]:
step = 1/6
step_by = np.arange(0, (2/3)+step, step)
step_by = step_by[step_by>=(1/6)]
step_by

array([0.16666667, 0.33333333, 0.5       , 0.66666667])

In [13]:
cycle_lengths = [6, 10, 30, 48, 60, 72, 90]
percent_ons = step_by
data_params = dict()
data_params["cycle_lengths"] = cycle_lengths
data_params["percent_ons"] = percent_ons
dc_tags = ss.get_list_of_dc_tags(cycle_lengths, percent_ons)
dc_tags

['30of30',
 '1of6',
 '2of6',
 '3of6',
 '4of6',
 '2of10',
 '3of10',
 '5of10',
 '7of10',
 '5of30',
 '10of30',
 '15of30',
 '20of30',
 '8of48',
 '16of48',
 '24of48',
 '32of48',
 '10of60',
 '20of60',
 '30of60',
 '40of60',
 '12of72',
 '24of72',
 '36of72',
 '48of72',
 '15of90',
 '30of90',
 '45of90',
 '60of90']

In [14]:
site_keys = ['Carp']
type_keys = ['LF', 'HF']
data_params["dc_tags"] = dc_tags
data_params['cur_dc_tag'] = '30of30'
data_params['metric_tag'] = 'bout_time_percentage'
data_params['recording_start'] = '00:00'
data_params['recording_end'] = '16:00'

In [15]:
for site_key in site_keys:
    for type_key in type_keys:
        print(site_key, type_key)
        data_params["site_tag"] = site_key
        data_params["site_name"] = SITE_NAMES[site_key]
        data_params["type_tag"] = type_key
        data_params["detector_tag"] = 'kd'
        file_paths = get_file_paths(data_params)

        activity_arr, btp_arr = comp.generate_activity_btp_for_dc_schemes_and_cont(data_params, file_paths, save=True)

Carp LF


  0%|          | 0/28 [00:00<?, ?it/s]

In [ ]:
data_params['metric_tag'] = 'call_rate'

for site_key in site_keys:
    for type_key in type_keys:
        print(site_key, type_key)
        data_params["site_tag"] = site_key
        data_params["site_name"] = SITE_NAMES[site_key]
        data_params["type_tag"] = type_key
        data_params["detector_tag"] = 'kd'
        file_paths = get_file_paths(data_params)

        activity_arr, callrate_arr = comp.generate_activity_call_rate_for_dc_schemes_and_cont(data_params, file_paths, save=True)

Carp LF


100%|██████████| 28/28 [01:41<00:00,  3.64s/it]


Carp HF


100%|██████████| 28/28 [00:28<00:00,  1.04s/it]


In [ ]:
data_params['metric_tag'] = 'activity_index'
data_params['index_time_block_in_secs'] = 5

for site_key in site_keys:
    for type_key in type_keys:
        print(site_key, type_key)
        data_params["site_tag"] = site_key
        data_params["site_name"] = SITE_NAMES[site_key]
        data_params["type_tag"] = type_key
        data_params["detector_tag"] = 'kd'
        file_paths = get_file_paths(data_params)

        activity_arr, actvtind_arr = comp.generate_activity_index_percent_for_dc_schemes_and_cont(data_params, file_paths, save=True)

Carp LF


100%|██████████| 28/28 [01:44<00:00,  3.75s/it]


Carp HF


100%|██████████| 28/28 [00:29<00:00,  1.07s/it]
